# Emerging Technologies - Problems

This notebook contains solutions to the problems assigned for the Emerging Technologies module.

**Author:** Tommy Mitchell  
**Module:** Emerging Technologies  
**Institution:** ATU

## Problem 1

*Problem description goes here.*